In [1]:
from utils.experiment import Experiment

In [2]:
exp = Experiment('point_mnist', 'swe', 'faiss-lsh', 9, ref_size=500, random_state=0, num_slices=10)
acc = exp.test()
print(acc)
print(exp.get_time_report())

loading dataset...
preprocess samples...
compute reference...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:05<00:00, 1953.19it/s]


(10000, 500)
0.9243
{'dataset': 'point_mnist', 'pooling': 'swe', 'ann': 'faiss-lsh', 'emb_time_per_sample': 0.000532407283782959, 'inf_time_per_sample': 0.0002918713092803955}


In [3]:
exp = Experiment('point_mnist', 'fs', 'faiss-lsh', 9, ref_size=500, random_state=0)
acc = exp.test()
print(acc)
print(exp.get_time_report())

loading dataset...
loading cached base embedding...
compute query embedding...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4302.50it/s]


(10000, 500)
0.7711
{'dataset': 'point_mnist', 'pooling': 'fs', 'ann': 'faiss-lsh', 'emb_time_per_sample': 0.00025090320110321044, 'inf_time_per_sample': 0.00028894672393798827}
